In [22]:
from tensorflow.keras.preprocessing import sequence
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPool1D, Dropout, SimpleRNN, LSTM, Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import sequence
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix

1. Pre-process the raw data

In [23]:
df = pd.read_csv('Epileptic Seizure Recognition.csv')
print(df.head().T)
df = df.drop('Unnamed', 1)
y_df = df['y']
print(y_df.head().T)
print(y_df.value_counts())
y_binary = pd.Series(np.where(df['y'].values == 1, 1, 0), df['y'].index)
print(y_binary.head().T)
print(y_binary.value_counts())
x_df = df.drop('y', 1)

/var/folders/pg/1pf8c0d10hs7bc4233k0d06h0000gp/T/ipykernel_63634/1474613655.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Unnamed', 1)
/var/folders/pg/1pf8c0d10hs7bc4233k0d06h0000gp/T/ipykernel_63634/1474613655.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_df = df.drop('y', 1)


2. Training.

In [24]:
test_recall = []
test_f1= []
test_acc = []


for i in range(30):
    under_sampler = RandomUnderSampler(random_state=1)
    X_res, y_res = under_sampler.fit_resample(x_df, y_binary)
    y_res.value_counts()

    X = []

    for i, row in X_res.iterrows():
        tmp_x = []
        for e in row.tolist():
            tmp_x.append([e])
        X.append(tmp_x)
        
    X = np.array(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y_res, test_size=0.2, random_state=1234)

    model = Sequential()
    model.add(Input(shape=(178,1)))
    model.add(LSTM(64,return_sequences=True))
    model.add(LSTM(32,return_sequences=True))
    model.add(LSTM(8))
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    print(model.summary())
    history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=2000, batch_size=256, verbose=2)

    y_pred1 = model.predict(X_test)
    y_pred = np.argmax(y_pred1, axis=1)

    # Print f1, precision, and recall scores
    test_acc.append((accuracy_score(y_test, y_pred)))
    test_recall.append((recall_score(y_test, y_pred)))
    test_f1.append((f1_score(y_test, y_pred)))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 178, 64)           16896     
                                                                 
 lstm_16 (LSTM)              (None, 178, 32)           12416     
                                                                 
 lstm_17 (LSTM)              (None, 8)                 1312      
                                                                 
 flatten_5 (Flatten)         (None, 8)                 0         
                                                                 
 dense_10 (Dense)            (None, 250)               2250      
                                                                 
 dense_11 (Dense)            (None, 2)                 502       
                                                                 
Total params: 33,376
Trainable params: 33,376
Non-trai

2023-02-06 22:22:06.896432: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:22:07.275758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:22:07.571785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:22:07.879252: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:22:08.442773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:22:08.988642: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:22:09.453957: I tensorflow/core/grappler/optimizers/cust

15/15 - 10s - loss: 0.6588 - accuracy: 0.7427 - val_loss: 0.5824 - val_accuracy: 0.8391 - 10s/epoch - 683ms/step
Epoch 2/30
15/15 - 3s - loss: 0.4925 - accuracy: 0.8614 - val_loss: 0.3559 - val_accuracy: 0.8826 - 3s/epoch - 169ms/step
Epoch 3/30
15/15 - 3s - loss: 0.3404 - accuracy: 0.8791 - val_loss: 0.2750 - val_accuracy: 0.9011 - 3s/epoch - 168ms/step
Epoch 4/30
15/15 - 2s - loss: 0.2854 - accuracy: 0.8940 - val_loss: 0.2596 - val_accuracy: 0.8935 - 2s/epoch - 167ms/step
Epoch 5/30
15/15 - 3s - loss: 0.2567 - accuracy: 0.9008 - val_loss: 0.2117 - val_accuracy: 0.9207 - 3s/epoch - 169ms/step
Epoch 6/30
15/15 - 3s - loss: 0.2172 - accuracy: 0.9196 - val_loss: 0.1951 - val_accuracy: 0.9272 - 3s/epoch - 169ms/step
Epoch 7/30
15/15 - 2s - loss: 0.2089 - accuracy: 0.9245 - val_loss: 0.1620 - val_accuracy: 0.9478 - 2s/epoch - 166ms/step
Epoch 8/30
15/15 - 3s - loss: 0.1812 - accuracy: 0.9372 - val_loss: 0.1463 - val_accuracy: 0.9424 - 3s/epoch - 169ms/step
Epoch 9/30
15/15 - 3s - loss: 0.1

2023-02-06 22:23:28.860200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:23:28.983234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:23:29.234612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-06 22:23:29.486152: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


29/29 [==============================] - 4s 93ms/step


In [25]:
test_recall = pd.DataFrame(test_recall, columns=['test_recall'])
test_f1 = pd.DataFrame(test_f1, columns=['test_f1'])
test_acc = pd.DataFrame(test_acc, columns=['test_acc'])

evaluation_result = pd.concat([test_recall, test_f1, test_acc],axis=1)
evaluation_result.to_csv('evaluation_baseline_LSTM.csv')

In [26]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()